In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [4]:
# Characters are sorted
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


In [26]:
print(chars[78])

z


In [28]:
# At index 0, insert a line break
chars.insert(0, "\0")

In [33]:
chars[79]

'z'

In [29]:
# At each character, join characters from index 1 to the 6th last index(79).
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [34]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [36]:
# The index of each character is packed as a dictionary
char_indices

{'\x00': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '=': 23,
 '?': 24,
 'A': 25,
 'B': 26,
 'C': 27,
 'D': 28,
 'E': 29,
 'F': 30,
 'G': 31,
 'H': 32,
 'I': 33,
 'J': 34,
 'K': 35,
 'L': 36,
 'M': 37,
 'N': 38,
 'O': 39,
 'P': 40,
 'Q': 41,
 'R': 42,
 'S': 43,
 'T': 44,
 'U': 45,
 'V': 46,
 'W': 47,
 'X': 48,
 'Y': 49,
 'Z': 50,
 '[': 51,
 ']': 52,
 '_': 53,
 'a': 54,
 'b': 55,
 'c': 56,
 'd': 57,
 'e': 58,
 'f': 59,
 'g': 60,
 'h': 61,
 'i': 62,
 'j': 63,
 'k': 64,
 'l': 65,
 'm': 66,
 'n': 67,
 'o': 68,
 'p': 69,
 'q': 70,
 'r': 71,
 's': 72,
 't': 73,
 'u': 74,
 'v': 75,
 'w': 76,
 'x': 77,
 'y': 78,
 'z': 79,
 '\x86': 80,
 '\xa4': 81,
 '\xa6': 82,
 '\xa9': 83,
 '\xab': 84,
 '\xc3': 85}

In [37]:
# Indices are mapped to characters
indices_char

{0: '\x00',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: "'",
 6: '(',
 7: ')',
 8: ',',
 9: '-',
 10: '.',
 11: '0',
 12: '1',
 13: '2',
 14: '3',
 15: '4',
 16: '5',
 17: '6',
 18: '7',
 19: '8',
 20: '9',
 21: ':',
 22: ';',
 23: '=',
 24: '?',
 25: 'A',
 26: 'B',
 27: 'C',
 28: 'D',
 29: 'E',
 30: 'F',
 31: 'G',
 32: 'H',
 33: 'I',
 34: 'J',
 35: 'K',
 36: 'L',
 37: 'M',
 38: 'N',
 39: 'O',
 40: 'P',
 41: 'Q',
 42: 'R',
 43: 'S',
 44: 'T',
 45: 'U',
 46: 'V',
 47: 'W',
 48: 'X',
 49: 'Y',
 50: 'Z',
 51: '[',
 52: ']',
 53: '_',
 54: 'a',
 55: 'b',
 56: 'c',
 57: 'd',
 58: 'e',
 59: 'f',
 60: 'g',
 61: 'h',
 62: 'i',
 63: 'j',
 64: 'k',
 65: 'l',
 66: 'm',
 67: 'n',
 68: 'o',
 69: 'p',
 70: 'q',
 71: 'r',
 72: 's',
 73: 't',
 74: 'u',
 75: 'v',
 76: 'w',
 77: 'x',
 78: 'y',
 79: 'z',
 80: '\x86',
 81: '\xa4',
 82: '\xa6',
 83: '\xa9',
 84: '\xab',
 85: '\xc3'}

In [38]:
# depending on the char_indices defined above, the entire text is parsed, and 
# the character is replaced with the index.
idx = [char_indices[c] for c in text]

In [52]:
# The indices of the first 10 characters are such
idx[:10]
print(idx[600901-4])

58


In [40]:
# As defined above, we have a mechanism to convert characters to indices and then back.
''.join(indices_char[i] for i in idx[:10])

'PREFACE\n\n\n'

In [41]:
# 3 character model. Given 3 characters, predict the fourth
cs=3
# xrange has 3 params : the start index, the end index and the incrementation of each element position. 
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

In [48]:
print(len(c1_dat))
print(len(c2_dat))
print(len(c3_dat))
print(type(c1_dat))

200299
200299
200299
<type 'list'>


In [44]:
c1_dat

[40,
 30,
 29,
 1,
 40,
 43,
 31,
 61,
 2,
 74,
 2,
 2,
 76,
 54,
 9,
 54,
 73,
 67,
 33,
 73,
 71,
 67,
 2,
 68,
 57,
 68,
 72,
 69,
 73,
 60,
 61,
 2,
 65,
 61,
 68,
 69,
 71,
 2,
 2,
 2,
 71,
 72,
 61,
 2,
 75,
 55,
 67,
 68,
 54,
 72,
 8,
 54,
 2,
 62,
 57,
 68,
 67,
 71,
 54,
 2,
 66,
 9,
 61,
 2,
 58,
 58,
 62,
 58,
 58,
 68,
 67,
 72,
 67,
 56,
 66,
 2,
 69,
 73,
 62,
 2,
 73,
 76,
 56,
 73,
 78,
 54,
 2,
 74,
 65,
 69,
 57,
 61,
 71,
 57,
 58,
 58,
 73,
 44,
 73,
 2,
 75,
 55,
 67,
 67,
 62,
 58,
 54,
 2,
 72,
 66,
 2,
 73,
 57,
 59,
 1,
 67,
 67,
 54,
 68,
 67,
 27,
 73,
 67,
 2,
 58,
 54,
 67,
 58,
 54,
 68,
 57,
 58,
 58,
 2,
 2,
 2,
 67,
 54,
 1,
 2,
 58,
 67,
 58,
 71,
 64,
 57,
 59,
 68,
 54,
 73,
 57,
 76,
 61,
 54,
 54,
 2,
 72,
 74,
 60,
 2,
 58,
 9,
 8,
 67,
 58,
 2,
 2,
 54,
 72,
 73,
 65,
 2,
 71,
 61,
 58,
 71,
 72,
 59,
 71,
 76,
 2,
 62,
 54,
 2,
 54,
 62,
 61,
 2,
 65,
 67,
 73,
 73,
 65,
 57,
 66,
 65,
 72,
 67,
 61,
 60,
 74,
 9,
 54,
 66,
 58,
 73,
 73,
 73,


In [45]:
c2_dat

[42,
 25,
 1,
 43,
 40,
 33,
 2,
 54,
 44,
 73,
 62,
 54,
 68,
 67,
 76,
 73,
 61,
 24,
 72,
 61,
 58,
 68,
 60,
 74,
 1,
 71,
 74,
 58,
 62,
 2,
 54,
 54,
 2,
 62,
 72,
 61,
 72,
 62,
 72,
 59,
 2,
 2,
 58,
 61,
 58,
 58,
 1,
 60,
 73,
 73,
 2,
 75,
 59,
 65,
 2,
 2,
 57,
 72,
 67,
 76,
 58,
 9,
 54,
 73,
 2,
 71,
 55,
 1,
 71,
 74,
 58,
 2,
 57,
 65,
 72,
 62,
 68,
 74,
 73,
 76,
 61,
 61,
 61,
 61,
 2,
 75,
 74,
 54,
 78,
 54,
 1,
 58,
 2,
 57,
 72,
 72,
 68,
 71,
 61,
 61,
 58,
 58,
 2,
 72,
 65,
 57,
 67,
 74,
 58,
 65,
 66,
 61,
 72,
 68,
 76,
 67,
 60,
 2,
 66,
 24,
 58,
 54,
 65,
 72,
 2,
 72,
 58,
 71,
 65,
 76,
 2,
 71,
 65,
 73,
 55,
 76,
 22,
 67,
 54,
 69,
 72,
 73,
 75,
 78,
 62,
 2,
 2,
 60,
 2,
 54,
 72,
 62,
 2,
 57,
 67,
 57,
 56,
 71,
 58,
 66,
 67,
 33,
 1,
 57,
 57,
 62,
 72,
 67,
 2,
 2,
 65,
 30,
 2,
 58,
 2,
 58,
 56,
 59,
 72,
 61,
 66,
 67,
 62,
 73,
 73,
 73,
 54,
 59,
 65,
 8,
 61,
 2,
 65,
 68,
 54,
 62,
 2,
 2,
 58,
 71,
 67,
 9,
 78,
 68,
 8,
 61,
 2,
 2,

In [46]:
c3_dat

[29,
 27,
 1,
 45,
 39,
 38,
 73,
 73,
 71,
 61,
 72,
 2,
 66,
 9,
 61,
 2,
 58,
 2,
 2,
 58,
 2,
 73,
 71,
 67,
 59,
 2,
 72,
 56,
 67,
 73,
 73,
 65,
 69,
 65,
 68,
 58,
 8,
 67,
 68,
 54,
 54,
 73,
 78,
 54,
 2,
 58,
 57,
 66,
 62,
 72,
 61,
 58,
 54,
 58,
 73,
 74,
 58,
 73,
 57,
 68,
 67,
 73,
 73,
 61,
 73,
 71,
 65,
 72,
 62,
 72,
 72,
 54,
 2,
 74,
 78,
 66,
 71,
 67,
 78,
 62,
 2,
 62,
 2,
 58,
 61,
 58,
 72,
 65,
 2,
 62,
 73,
 62,
 54,
 71,
 72,
 2,
 2,
 74,
 8,
 54,
 2,
 58,
 74,
 64,
 65,
 2,
 57,
 67,
 58,
 78,
 58,
 68,
 2,
 71,
 62,
 62,
 2,
 76,
 54,
 2,
 71,
 62,
 78,
 61,
 61,
 2,
 75,
 2,
 65,
 58,
 61,
 72,
 59,
 68,
 58,
 68,
 2,
 57,
 73,
 71,
 58,
 2,
 58,
 2,
 67,
 68,
 57,
 66,
 72,
 67,
 2,
 73,
 72,
 2,
 57,
 62,
 68,
 54,
 57,
 62,
 9,
 30,
 62,
 58,
 8,
 73,
 73,
 57,
 54,
 54,
 3,
 68,
 73,
 71,
 54,
 2,
 68,
 58,
 2,
 68,
 54,
 73,
 67,
 61,
 2,
 1,
 72,
 54,
 58,
 2,
 54,
 54,
 2,
 60,
 2,
 58,
 68,
 73,
 2,
 68,
 57,
 67,
 2,
 71,
 2,
 54,
 62,
 62,
 5

In [47]:
c4_dat

[30,
 29,
 1,
 40,
 43,
 31,
 61,
 2,
 74,
 2,
 2,
 76,
 54,
 9,
 54,
 73,
 67,
 33,
 73,
 71,
 67,
 2,
 68,
 57,
 68,
 72,
 69,
 73,
 60,
 61,
 2,
 65,
 61,
 68,
 69,
 71,
 2,
 2,
 2,
 71,
 72,
 61,
 2,
 75,
 55,
 67,
 68,
 54,
 72,
 8,
 54,
 2,
 62,
 57,
 68,
 67,
 71,
 54,
 2,
 66,
 9,
 61,
 2,
 58,
 58,
 62,
 58,
 58,
 68,
 67,
 72,
 67,
 56,
 66,
 2,
 69,
 73,
 62,
 2,
 73,
 76,
 56,
 73,
 78,
 54,
 2,
 74,
 65,
 69,
 57,
 61,
 71,
 57,
 58,
 58,
 73,
 44,
 73,
 2,
 75,
 55,
 67,
 67,
 62,
 58,
 54,
 2,
 72,
 66,
 2,
 73,
 57,
 59,
 1,
 67,
 67,
 54,
 68,
 67,
 27,
 73,
 67,
 2,
 58,
 54,
 67,
 58,
 54,
 68,
 57,
 58,
 58,
 2,
 2,
 2,
 67,
 54,
 1,
 2,
 58,
 67,
 58,
 71,
 64,
 57,
 59,
 68,
 54,
 73,
 57,
 76,
 61,
 54,
 54,
 2,
 72,
 74,
 60,
 2,
 58,
 9,
 8,
 67,
 58,
 2,
 2,
 54,
 72,
 73,
 65,
 2,
 71,
 61,
 58,
 71,
 72,
 59,
 71,
 76,
 2,
 62,
 54,
 2,
 54,
 62,
 61,
 2,
 65,
 67,
 73,
 73,
 65,
 57,
 66,
 65,
 72,
 67,
 61,
 60,
 74,
 9,
 54,
 66,
 58,
 73,
 73,
 73,
 72,


In [53]:
# Inputs are stacked
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [58]:
len(x3)

200297

In [59]:
# This is the output.
y = np.stack(c4_dat[:-2])

In [60]:
len(y)

200297

In [61]:
# First four values of each of the above defined input numpy arrays
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [62]:
# The first four values of the output array
y[:4]

array([30, 29,  1, 40])

In [63]:
# The input shape and the output shapes are the same as expected. We have the same number of outputs as inputs!
x1.shape, y.shape

((200297,), (200297,))

In [64]:
# The number of latent factors(size of the embedding matrix.)
n_fac = 42

In [70]:
vocab_size

86

In [74]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    # Embeddings turn indices into dense vectors.
    # Thus, here, we are passing in n_in = 86 and outputing a dense vector of 42 latent factors
    # and then flattening that
    print(inp)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    print(emb)
    return inp, Flatten()(emb)

In [75]:
# Creating embeddings for each of the inputs c1, c2 & c3
# 86 index values are given 42 latent factors
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

c1
Reshape{3}.0
c2
Reshape{3}.0
c3
Reshape{3}.0


In [76]:
# The hidden state is given an arbitrary size of 256
n_hidden = 256

In [77]:
# This is the input from an input matrix to a hidden state
# dense_in will produce an output shape of (*, 256) and apply an activation of relu
dense_in = Dense(n_hidden, activation='relu')

In [78]:
# The activation of the first hidden layer is simply the dense_in function
# applied to the result of the emebedding of the first character.
c1_hidden = dense_in(c1)

In [79]:
# This is the input from the first hidden state to the next hidden state.
# The output of the first hidden layer is also 256 dimensions.
dense_hidden = Dense(n_hidden, activation='tanh')

In [80]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

In [81]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

In [82]:
# The hidden state is then transformed to an output(prediction of vocab_size dimension)
dense_out = Dense(vocab_size, activation='softmax')

In [83]:
c4_out = dense_out(c3_hidden)

In [84]:
# The model is defined by the functional API with an input of an list of 3 elements and and output of one
model = Model([c1_in, c2_in, c3_in], c4_out)

In [85]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = Adam())

In [86]:
model.optimizer.lr = 0.000001

In [87]:
# We pass in the numpy arrays of inputs & outputs to fit the model to the data.
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 14s - loss: 4.4190    
Epoch 2/4
200297/200297 [==============================] - 14s - loss: 4.2980    
Epoch 3/4
200297/200297 [==============================] - 14s - loss: 4.0389    
Epoch 4/4
200297/200297 [==============================] - 14s - loss: 3.6469    


In [95]:
model.optimizer.lr = 0.01
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=10)

Epoch 1/10
200297/200297 [==============================] - 14s - loss: 3.0723    
Epoch 2/10
200297/200297 [==============================] - 14s - loss: 3.0655    
Epoch 3/10
200297/200297 [==============================] - 14s - loss: 3.0587    
Epoch 4/10
200297/200297 [==============================] - 14s - loss: 3.0519    
Epoch 5/10
200297/200297 [==============================] - 14s - loss: 3.0450    
Epoch 6/10
200297/200297 [==============================] - 14s - loss: 3.0378    
Epoch 7/10
200297/200297 [==============================] - 14s - loss: 3.0305    
Epoch 8/10
200297/200297 [==============================] - 14s - loss: 3.0229    
Epoch 9/10
200297/200297 [==============================] - 14s - loss: 3.0150    
Epoch 10/10
200297/200297 [==============================] - 14s - loss: 3.0069    


In [96]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [97]:
get_next('phi')

' '

In [94]:
get_next(' an')

' '

In [71]:
# FIRST RNN
cs = 8
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)] for n in range(cs)]

In [72]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [73]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [74]:
len(xs), xs[0].shape

(8, (75110,))

In [75]:
y = np.stack(c_out_dat[:-2])

In [76]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [77]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [78]:
# Number of latent factors
n_fac = 42

In [80]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [81]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [82]:
n_hidden = 256

In [83]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [84]:
hidden= dense_in(c_ins[0][1])

In [86]:
for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [87]:
c_out = dense_out(hidden)

In [88]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [89]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

Epoch 1/12
75110/75110 [==============================] - 10s - loss: 2.5366    
Epoch 2/12
75110/75110 [==============================] - 10s - loss: 2.2559    
Epoch 3/12
75110/75110 [==============================] - 10s - loss: 2.1520    
Epoch 4/12
75110/75110 [==============================] - 10s - loss: 2.0811    
Epoch 5/12
75110/75110 [==============================] - 10s - loss: 2.0270    
Epoch 6/12
75110/75110 [==============================] - 10s - loss: 1.9803    
Epoch 7/12
75110/75110 [==============================] - 10s - loss: 1.9426    
Epoch 8/12
75110/75110 [==============================] - 10s - loss: 1.9100    
Epoch 9/12
75110/75110 [==============================] - 10s - loss: 1.8811    
Epoch 10/12
75110/75110 [==============================] - 10s - loss: 1.8537    
Epoch 11/12
75110/75110 [==============================] - 10s - loss: 1.8280    
Epoch 12/12
75110/75110 [==============================] - 10s - loss: 1.8055    


In [90]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [91]:
get_next('for thos')

'e'

In [93]:
get_next('queens o')

'f'

In [96]:
get_next('part of ')

't'

In [98]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [99]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [100]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_7 (Embedding)          (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 256)           76544       embedding_7[0][0]                
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 86)            22102       simplernn_1[0][0]                
Total params: 102258
____________________________________________________________________________________________________


In [101]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [102]:
model.fit(np.stack(xs,1), y, batch_size=64, nb_epoch=8)

Exception: Error when checking model input: expected embedding_input_1 to have 2 dimensions, but got array with shape (75110, 8, 1)